In [ ]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap

In [ ]:
import ee
# ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
import geemap
import ipyleaflet
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import glob

In [ ]:
# define directory
out_dir = os.getcwd()
bucket_name = 'cities-indicators'
aws_s3_dir = "https://"+bucket_name+".s3.eu-west-3.amazonaws.com"
boundary_ext = '/data/boundaries/'
indicators_file_aws = 'indicators/indicators.csv'

# Load data

In [ ]:
# get list of cities
boundary_georef = pd.read_csv(aws_s3_dir + boundary_ext + 'boundary_georef.csv')
boundary_georef

# Extract vegetation & water change layers by city

In [ ]:
## Functions to calculate vegetation and water cover raster layers

NDVIthreshold = 0.4 # decimal
NDWIthreshold = 0.3 # decimal
halfpvalue = 0.025 # half the value of the p-value threshold to be used in the significance test. 
MAX_CLOUD_PROB=30
date_start = '2016-01-01'
date_end = '2022-12-31'


S2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
S2C = ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY")

## get cloudmasked image collection 

def mask_and_count_clouds(s2wc,geom):
    s2wc=ee.Image(s2wc)
    geom=ee.Geometry(geom.geometry())
    is_cloud=ee.Image(s2wc.get('cloud_mask')).gt(MAX_CLOUD_PROB).rename('is_cloud')
    nb_cloudy_pixels=is_cloud.reduceRegion(
        reducer=ee.Reducer.sum().unweighted(), 
        geometry=geom, 
        scale=10, 
        maxPixels=1e9
   )
    return s2wc.updateMask(is_cloud.eq(0)).set('nb_cloudy_pixels',nb_cloudy_pixels.getNumber('is_cloud')).divide(10000)

def mask_clouds_and_rescale(im):
    clouds=ee.Image(im.get('cloud_mask')).select('probability')
    return im.updateMask(clouds.lt(MAX_CLOUD_PROB)).divide(10000)

def get_masked_s2_collection(roi,start,end):
    criteria=(ee.Filter.And(
            ee.Filter.date(start,end),
            ee.Filter.bounds(roi)
        ))
    s2=S2.filter(criteria).select('B2','B3','B4','B8','B11','B12')
    s2c=S2C.filter(criteria)
    s2_with_clouds=(ee.Join.saveFirst('cloud_mask').apply(**{
        'primary': ee.ImageCollection(s2),
        'secondary': ee.ImageCollection(s2c),
        'condition': ee.Filter.equals(**{'leftField':'system:index','rightField':'system:index'}) 
        }))
    def _mcc(im):
        return mask_and_count_clouds(im,roi) 
    #s2_with_clouds=ee.ImageCollection(s2_with_clouds).map(_mcc)
    #s2_with_clouds=s2_with_clouds.limit(image_limit,'nb_cloudy_pixels')
    s2_with_clouds=ee.ImageCollection(s2_with_clouds).map(mask_clouds_and_rescale)#.limit(image_limit,'CLOUDY_PIXEL_PERCENTAGE')
    return  ee.ImageCollection(s2_with_clouds)


## annual image collections and images

def AnnualIC(ic,year):
    def addNDVI(image):
        ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
        return image.addBands(ndvi)
    def addNDWI(image):
        ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
        return image.addBands(ndwi)
    def addyear(image):
        return image.set("year",year)
    # ImgColl = ic.filterDate(''+str(year)+'-01-01', ''+str(year)+'-12-31').map(addNDVI).map(addNDWI).map(addyear)
    ImgColl = ic.filter(ee.Filter.stringStartsWith('system:index',year)).map(addNDVI).map(addNDWI).map(addyear)
    return ImgColl

def AnnualImgGreen(ic,year):
    # greenest = ic.qualityMosaic('NDVI').select('NDVI').addBands(ee.Image(year).rename('time_start')).float()
    greenest = ic.select('NDVI').reduce(ee.Reducer.median()).rename('NDVI').addBands(ee.Image(year).rename('time_start')).float()
    greenestThres = greenest.updateMask(greenest.select('NDVI').gte(NDVIthreshold))
    bluest = ic.qualityMosaic('NDWI').select('NDWI').addBands(ee.Image(year).rename('time_start')).float()
    # bluest = ic.select('NDWI').reduce(ee.Reducer.median()).rename('NDWI').addBands(ee.Image(year).rename('time_start')).float()
    bluestThres = bluest.updateMask(bluest.select('NDWI').gte(NDWIthreshold))
    greenestThresnowater = greenestThres.updateMask(bluestThres.select('NDWI').unmask().Not())
    greenestnowater = greenest.updateMask(bluestThres.select('NDWI').unmask().Not())
    return greenestnowater
def AnnualImgWater(ic,year):
    # bluest = ic.qualityMosaic('NDWI').select('NDWI').addBands(ee.Image(year).rename('time_start')).float()
    bluest = ic.select('NDWI').reduce(ee.Reducer.median()).rename('NDWI').addBands(ee.Image(year).rename('time_start')).float()
    greenest = ic.qualityMosaic('NDVI').select('NDVI').addBands(ee.Image(year).rename('time_start')).float()
    greenestThres = greenest.updateMask(greenest.select('NDVI').gte(NDVIthreshold))
    bluestnowater = bluest.updateMask(greenestThres.select('NDVI').unmask().Not())
    return bluest
def AnnualImgWatermask(ic,year):
    bluest = ic.qualityMosaic('NDWI').select('NDWI').addBands(ee.Image(year).rename('time_start')).float()
    # bluest = ic.select('NDWI').reduce(ee.Reducer.median()).rename('NDWI').addBands(ee.Image(year).rename('time_start')).float()
    bluestThres = bluest.updateMask(bluest.select('NDWI').gte(NDWIthreshold))
    return bluestThres
def AnnualImgGreenmask(ic,year):
    greenest = ic.qualityMosaic('NDVI').select('NDVI').addBands(ee.Image(year).rename('time_start')).float()
    # greenest = ic.select('NDVI').reduce(ee.Reducer.median()).rename('NDVI').addBands(ee.Image(year).rename('time_start')).float()
    greenestThres = greenest.updateMask(greenest.select('NDVI').gte(NDVIthreshold))
    return greenestThres


In [ ]:
# Functions for significance test
# https://developers.google.com/earth-engine/tutorials/community/nonparametric-trends
# https://code.earthengine.google.com/bce3dc00c56df4246c5d32f5fcccf5c7
#//////////////////////////////////////singificance test///////////////////////////////////////////////////////////////

    
def senum(lfx):
    def senumwrap(Im):
        esty=lfx.select('scale').multiply(Im.select('time_start')).add(lfx.select('offset'))
        diff=Im.select('NDVI').subtract(esty)
        pow=diff.multiply(diff)
        return (pow)
    return (senumwrap)
    
def sedenom(mosaicmeanx):
    def sedenomwrap(Im):
        diff=Im.select('time_start').subtract(mosaicmeanx.select('time_start'));
        pow=diff.multiply(diff);
        return (pow)
    return (sedenomwrap)
    
# https://en.wikipedia.org/wiki/Error_function#Cumulative_distribution_function
def eeCdf(t):
    return ee.Image(0.5).multiply(ee.Image(1).add(ee.Image(t).divide(ee.Image(2).sqrt()).erf()));

def invCdf(p):
    return ee.Image(2).sqrt().multiply(ee.Image(p).multiply(2).subtract(1).erfInv());

#/////green normalized Difference///////
def significanceGreen(gtrend,glinearfit,SampleNumber): 
    


    mosaicmean=gtrend.mean();
    mosaicNum=gtrend.map(senum(glinearfit)).sum();
    mosaicDenom=gtrend.map(sedenom(mosaicmean)).sum();
    StdDev=mosaicNum.divide(mosaicDenom);
    #Standard Error - from SampleNumber samples
    se= StdDev.divide(SampleNumber).sqrt();
    #Test Statistic
    gT=glinearfit.select('scale').divide(se);
    # Compute P-values.
    gP = ee.Image(1).subtract(eeCdf(gT.abs()));
    # Pixels that can have the null hypothesis (there is no trend) rejected.
    # Specifically, if the true trend is zero, there would be less than 5%
    # chance of randomly obtaining the observed result (that there is a trend).
    # glf=glfLimit.updateMask(gP.lte(0.025));  
    # Map.addLayer(glf, {'bands':['scale'],'min': -1, 'max': 1,'palette':['red','yellow','green']}, 'significant veg trends');
    return gP

    
#/////water normalized Difference///////
def significanceWater(wtrend,wlinearfit,SampleNumber): 
    
    def senum(lfx):
        def senumwrap(Im):
            esty=lfx.select('scale').multiply(Im.select('time_start')).add(lfx.select('offset'))
            diff=Im.select('NDWI').subtract(esty)
            pow=diff.multiply(diff)
            return (pow)
        return (senumwrap)

    mosaicmean=wtrend.mean();
    mosaicNum=wtrend.map(senum(wlinearfit)).sum();
    mosaicDenom=wtrend.map(sedenom(mosaicmean)).sum();
    StdDev=mosaicNum.divide(mosaicDenom);
    #Standard Error - from SampleNumber samples
    se= StdDev.divide(SampleNumber).sqrt();
    #Test Statistic
    wT=wlinearfit.select('scale').divide(se);
    # Compute P-values.
    wP = ee.Image(1).subtract(eeCdf(wT.abs()));
    # Pixels that can have the null hypothesis (there is no trend) rejected.
    # Specifically, if the true trend is zero, there would be less than 5%
    # chance of randomly obtaining the observed result (that there is a trend).
    # wlf=wlfLimit.updateMask(wP.lte(0.025));
    return wP

# Map
#//----------------------------------------------------------------------------------------//


In [ ]:
# function to generate vegetation and water trend and change maps
def get_map_vegwaterchange(IC):
    
    gwic2018 = AnnualIC(IC,'2018')
    gwic2019 = AnnualIC(IC,'2019')
    gwic2020 = AnnualIC(IC,'2020')
    gwic2021 = AnnualIC(IC,'2021')
    gwic2022 = AnnualIC(IC,'2022')
    gwic = (
        # gwic2018.merge
        (gwic2019).merge(gwic2020).merge(gwic2021).merge(gwic2022))
    # print(gwic.first().getInfo())

    d = {}
    for i in range(2019,2023):
        # https://stackoverflow.com/questions/6181935/how-do-you-create-different-variable-names-while-in-a-loop
        filteredgwic = gwic.filter(ee.Filter.eq("year",str(i)))
        d["g{0}".format(i)] = AnnualImgGreen(filteredgwic,i)
        d["w{0}".format(i)] = AnnualImgWater(filteredgwic,i)
        d["w{0}mask".format(i)] = AnnualImgWatermask(filteredgwic,i)
        d["g{0}mask".format(i)] = AnnualImgGreenmask(filteredgwic,i)
    
    gtrend = ee.ImageCollection.fromImages([#d["g2018"],
                                            d["g2019"],d["g2020"],d["g2021"],d["g2022"]])
    wtrend = ee.ImageCollection.fromImages([#d["w2018"],
                                            d["w2019"],d["w2020"],d["w2021"],d["w2022"]])
    wanyyear = (ee.Image(
        #d["w2018mask"].select('NDWI')).blend(
        d["w2019mask"].select('NDWI'))
        .blend(d["w2020mask"].select('NDWI'))
        .blend(d["w2021mask"].select('NDWI'))
        .blend(d["w2022mask"].select('NDWI'))
                )
    ganyyear = (ee.Image(
        #d["g2018mask"].select('NDVI')).blend(
        d["g2019mask"].select('NDVI'))
        .blend(d["g2020mask"].select('NDVI'))
        .blend(d["g2021mask"].select('NDVI'))
        .blend(d["g2022mask"].select('NDVI'))
                )
    startyear = ee.Number(2019)
    endyear = ee.Number(2022)
    sampleNumber = (endyear.subtract(startyear)).add(1)
    gstartmask = AnnualImgGreenmask(gwic.filter(ee.Filter.eq("year",'2019')),startyear)
    wstartmask = AnnualImgWatermask(gwic.filter(ee.Filter.eq("year",'2019')),startyear)
    gendmask = AnnualImgGreenmask(gwic.filter(ee.Filter.eq("year",'2022')),endyear)
    wendmask = AnnualImgWatermask(gwic.filter(ee.Filter.eq("year",'2022')),endyear)   
    
    minSlopeVeg = ee.Number(0.1) #0.05 #   minimum threshold scale value from linear fit trend line to be considered a vegetation change. 
    minSlopeWater = ee.Number(0.1) # 0.05 #  minimum threshold scale value from linear fit trend line to be considered a water change. 

    # Create linear fit trend line for years of NDVI data
    # https://developers.google.com/earth-engine/guides/reducers_regression#linearfit
    glinearfit = ee.Image(gtrend.select(['time_start','NDVI']).reduce(ee.Reducer.linearFit()));

    ## apply significance test
    # Pixels that can have the null hypothesis (there is no trend) rejected.
    # Specifically, if the true trend is zero, there would be less than 5% (double "halfpvalue")
    # chance of randomly obtaining the observed result (that there is a trend).
    gsignifMask = significanceGreen(gtrend,glinearfit,sampleNumber).lte(halfpvalue)
    glinearfit = glinearfit.updateMask(gsignifMask)

    # Annual slope value for each pixel above threshold. If interested in value for timeperiod, can use .multiply(ee.Image(6)). Can also mask based on offset to limit based on starting vegetation level. 
    glfLimit = (glinearfit.select('scale')#.multiply(ee.Image(6))
                .updateMask(glinearfit.select('scale').gte(minSlopeVeg).Or(glinearfit.select('scale').lte(ee.Number(0).subtract(minSlopeVeg))))
                #.updateMask(glinearfit.select('offset').gte(-0.1))
               )
    greenanyyearMask = ee.Image(0).where(ganyyear.neq(0),1)
    # Map.addLayer(greenanyyearMask,{},'green any year mask')

    glfLimitanyyeargreen = glfLimit.updateMask(greenanyyearMask)
    glfLimitanyyeargreenLoss = glfLimitanyyeargreen.updateMask(glfLimitanyyeargreen.lt(0))
    glfLimitanyyeargreenGain = glfLimitanyyeargreen.updateMask(glfLimitanyyeargreen.gt(0))

    # Map.addLayer(glfLimitanyyeargreen,{"min": -0.3, "max": 0.3, "palette":["red","yellow","green"]},"Veg Trend");

    # Create linear fit trend line for years of NDWI data
    wlinearfit = ee.Image(wtrend.select(['time_start','NDWI']).reduce(ee.Reducer.linearFit()));
    
    ## apply significance test
    # Pixels that can have the null hypothesis (there is no trend) rejected.
    # Specifically, if the true trend is zero, there would be less than 5% (double "halfpvalue")
    # chance of randomly obtaining the observed result (that there is a trend).
    wsignifMask = significanceWater(wtrend,wlinearfit,sampleNumber).lte(halfpvalue)
    wlinearfit = wlinearfit.updateMask(wsignifMask)
    
    # Annual slope value for each pixel above threshold. If interested in value for timeperiod, can use .multiply(ee.Image(6)). Can also mask based on offset to limit based on starting vegetation level. 
    wlfLimit = (wlinearfit.select('scale')#.multiply(ee.Image(6))
                .updateMask(wlinearfit.select('scale').gte(minSlopeWater).Or(wlinearfit.select('scale').lte(ee.Number(0).subtract(minSlopeVeg))))
                #.updateMask(wlinearfit.select('offset').gte(-0.1))
               )

    wateranyyearMask = ee.Image(0).where(wanyyear.neq(0),1)
    # Map.addLayer(wateranyyearMask,{},'water any year mask')

    wlfLimitanyyearwater = wlfLimit.updateMask(wateranyyearMask)
    wlfLimitanyyearwaterLoss = wlfLimitanyyearwater.updateMask(wlfLimitanyyearwater.lt(0))
    wlfLimitanyyearwaterGain = wlfLimitanyyearwater.updateMask(wlfLimitanyyearwater.gt(0))

    # Map.addLayer(wlfLimitanyyearwater,{"min": -0.3, "max": 0.3, "palette":['brown','white','blue']},"Water Trend");
    #Map.addLayer(wlfLimitanyyearwaterLoss,{"min": -0.3, "max": 0.3, "palette":['brown','white','blue']},"Water Trend Loss");
    #Map.addLayer(wlfLimitanyyearwaterGain,{"min": -0.3, "max": 0.3, "palette":['brown','white','blue']},"Water Trend Gain");

    gorwstartmask = gstartmask.select('NDVI').blend(wstartmask.select('NDWI'))
    gorwendmask = gendmask.select('NDVI').blend(wendmask.select('NDWI'))
    greenorwaterLimitLoss = glfLimitanyyeargreenLoss.blend(wlfLimitanyyearwaterLoss)
    greenorwaterLimitGain = glfLimitanyyeargreenGain.blend(wlfLimitanyyearwaterGain)
    combinedStartLossGain = gorwstartmask.rename('startgreenwaterIndex').addBands(gorwendmask.rename('endgreenwaterIndex')).addBands(greenorwaterLimitLoss.rename('lossgreenwaterSlope')).addBands(greenorwaterLimitGain.rename('gaingreenwaterSlope'))

    return combinedStartLossGain
    # Map.addLayer(gorw2016mask,{"min": -0.3, "max": 0.3, "palette":['brown','white','blue']},"2016 green or water");
    # Map.addLayer(greenorwaterLimitLoss,{"min": -0.3, "max": 0.3, "palette":['brown','white','blue']},"Green or water Loss");
    # Map.addLayer(greenorwaterLimitGain,{"min": -0.3, "max": 0.3, "palette":['brown','white','blue']},"Green or water Gain");


In [ ]:
for i in range(0,len(boundary_georef)):
    print(i)
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    print(boundary_id)
    
    # read boundaries
    boundary_path = aws_s3_dir + boundary_ext +'boundary-'+boundary_id+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    ## apply functions
    s2cloudmasked = get_masked_s2_collection(boundary_geo_ee,date_start,date_end)
    vegwatermap = get_map_vegwaterchange(s2cloudmasked)
    vegwaterstartmap = vegwatermap.select('startgreenwaterIndex')
    vegwaterendmap = vegwatermap.select('endgreenwaterIndex')
    vegwaterchangemap = vegwatermap.select('lossgreenwaterSlope').blend(vegwatermap.select('gaingreenwaterSlope')).rename('changegreenwaterSlope')

    # Download ee.Image of veg/water change as GeoTIFF
    geemap.ee_export_image_to_drive(
        vegwaterchangemap.toFloat(),  # keep as float to better represent slope values. Alternatively could convert to integer to reduce file size: .multiply(100).round().toByte()
        description = boundary_id + '-VegWater-change',
        folder='data', 
        scale=50, # 10 for native resolution, 50 for reduced file size
        region=boundary_geo_ee.geometry(),
        maxPixels = 5000000000
    )

In [ ]:
# ## create map
# Map = geemap.Map(height="400px")
# ## add basemap and center on area of interest
# Map.add_basemap('HYBRID')
# Map.centerObject(boundary_geo_ee,11)

# Map.addLayer(vegwaterstartmap,{'palette':['white','green'],'min':0,'max':1},'vegetation or water in start year')
# Map.addLayer(vegwaterendmap,{'palette':['white','green'],'min':0,'max':1},'vegetation or water in end year')
# Map.addLayer(vegwaterchangemap,{'palette':['#c75d9d','#96bc3e'],'min':-0.15,'max':0.15},'vegetation or water change')
# Map

# Upload in aws

Since we can't download directly the rasters locally due to their size, the rasters are stored in a google-drive folder and then downloaded locally in order to push them back to s3 bucket.

In [ ]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [ ]:
for i in range(0, len(boundary_georef)):
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n boundary_id: "+boundary_id)

    # read local raster
    city_file = 'data/' + boundary_id +'-VegWater-change.tif'
    raster_path = os.path.join(out_dir, city_file)
    
    # upload in s3
    s3.meta.client.upload_file(raster_path, 
                               bucket_name, 
                               'data/vegetation/sentinel-2/'+ boundary_id + '-VegWater-change-2019to2022-50m.tif',
                               ExtraArgs={'ACL':'public-read'})